In [1]:
import numpy as np
from pathlib import Path
import cv2
import pandas as pd
from skimage import measure, filters, draw
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import tqdm 
from scipy import ndimage

import SimpleITK as sitk
import itkwidgets 
import napari
import copy

from napari.utils.colormaps import colormap_utils as cu

c:\Users\zoeba\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
def fit_ellipses(mask, rangemin, rangemax):
    ''' 
    Le but est de récuperer les paramètres d'une liste ellipse en fonction d'une liste de masques fournis par le network construction
    Args :
        mask (numpy array) : masque des artères
        rangemin (int): début des coupes sélectionnées
        rangemax (int): fin des coupes sélectionnées
    Return:
        list_params_ellispses (list[list[double]]): paramètres de chaque ellipse
    '''
    # list_params_ellipses=[]
    # for img in tqdm.tqdm(mask[rangemin:rangemax]):
    #     list_points_contour=[]
    #     for j in range(img.shape[0]):
    #         for i in range(img.shape[1]):
    #             if img[j][i] == 1:
    #                 list_points_contour.append((i,j))
    #     array_points_contour=np.array(list_points_contour)
    #     ell = measure.EllipseModel()
    #     #draw contour
    #     #cv2.fitellipse()
    #     ell.estimate(array_points_contour)
    #     list_params_ellipses.append(ell.params)
    # return list_params_ellipses

In [4]:
def create_3d_ellipses(list_params_ellipses):
    '''
    Crée le numpy array représentant les images des ellipses
    Args:
        list_params_ellispses (list[list[double]]): paramètres de chaque ellipse
    Return:
        ellipses_3d (numpy array) : 3d images of the ellipses
    '''
    # list_2D_ellipses = []
    # for params in list_params_ellipses:
    #     xc, yc, a, b, theta = params
    #     t = np.linspace(0, 2*np.pi, 100)
    #     X = xc + a*np.cos(theta)*np.cos(t) - b*np.sin(theta)*np.sin(t)
    #     Y = yc + a*np.sin(theta)*np.cos(t) + b*np.cos(theta)*np.sin(t)
    #     built_ellipse = np.zeros((512,512))
    #     for i in range(len(X)):
    #         built_ellipse[int(Y[i])][int(X[i])] = 1

    #     list_2D_ellipses.append(built_ellipse)
    # ellipses_3D = np.array(list_2D_ellipses)
    # return ellipses_3D

In [19]:
def find_slice_with_mask(mask):
    '''
    Détecte sur les index des premières et dernière tranches d'un masque. 
    Args:
        mask (np array): le masque ou le fond est des 1 et le masque des 0
    Return:
        (rangemin,rangemax) (int,int): tuple avec l'index des premières et dernière tranches d'un masque. 
    '''
    nb_slices_with_masks=[]
    for nb_slice in range(mask.shape[0]-1, 0,-1):
        if 1 in mask[nb_slice]:
            nb_slices_with_masks.append(nb_slice)
    rangemin = np.min(nb_slices_with_masks) #mask 1
    rangemax = np.max(nb_slices_with_masks) #mask 1
    return (rangemin,rangemax)

In [8]:
def construct_ellipses_network(mask_list,range_to_do):
    '''
    En partant de la liste de masques construite par la network construction, des ellipses sont approximées pour reconstruire un réseau artériel lisse. 
    Args:
        mask_list (list[np.array]): la liste des masque construite par la network construction
    Return:
        info_all_masks (list[dic]): une liste où pour chaque loigne (qui correspond à un masque), il y a un dictionnaire où les clefs sont: rangemin, rangemax, mask,  smooth_ellipses
        ellipses_network (np.array): l'image 3D où le réseau d'ellipse est reconstitué
    ''' 
    info_all_masks = []
    ellipses_network=np.zeros(mask_list[0].shape)
    for index_mask in (range(range_to_do)):#len(mask_list))):
        print(index_mask+1, " out of ", range_to_do)
        mask = mask_list[index_mask]
        mask_01 = np.where(mask == 255, 1, 0)
        rangemin,rangemax=find_slice_with_mask(mask_01)

        # list_params = fit_ellipses(mask_01, rangemin, rangemax)
        #[0] centre [1] WIDTH HEIGHT 
        # fit = np.polyfit(list_params[1][0] * list_params[1][1])
        # predict_size_area = np.poly1d(fit)
        # predict_size_area(1380)
    #     seuil = [7, 7, 60, 60, 300]
    #     ponderation = [1,2,3,4,4,4,3,2,1]
    #     nb_slices = 9

    #     homogeneous_params = ensuring_3d_continuity(list_params, seuil)
    #     smooth_params = smooth_3d_ellipses(homogeneous_params, nb_slices, ponderation)
    #     smooth_ellipses = create_3d_ellipses(smooth_params)

    #     before=np.zeros((rangemin, mask_01.shape[1],mask_01.shape[1]))
    #     after=np.zeros((mask_01.shape[0]-rangemax, mask_01.shape[1],mask_01.shape[1]))
    #     whole=np.concatenate((before,smooth_ellipses,after))
        
    #     ellipses_network=ellipses_network+whole

    #     ellipses_network_ok=np.where(ellipses_network>1,1,ellipses_network)

    #     info_of_one_mask={'index':index_mask, 'rangemin':rangemin,'rangemax':rangemax,'mask':mask_01, 'smooth_ellipses':smooth_ellipses}
    #     info_all_masks.append(info_of_one_mask)
    # return info_all_masks, ellipses_network_ok

In [9]:
def construct_mask_network(mask_list,range_to_do):
    '''
 
    ''' 
    mask_network=np.zeros(mask_list[0].shape)
    for index_mask in tqdm.tqdm(range(range_to_do)):#len(mask_list))):
        mask_ok=np.where(mask_list[index_mask]==255,0,1)
        
        mask_network=mask_network+mask_ok
        mask_network_ok=np.where(mask_network>1,1,mask_network)
    return mask_network_ok

In [10]:
def filling_ellipses(ellipses):
    # #avec fill binary holes
    # '''
    # En partant  des images 3D des ellipses, on crée une image 3D des ellipses remplies. 
    # Args:
    #     ellipses (np.array): les ellipses à remplir
    # Return:
    #     filled_ellipses_np (np.array): les ellipses remplies
    # ''' 
    # filled_ellipses=[]
    # for index in range(len(ellipses)):
    #     ellipse=copy.deepcopy(ellipses[index]) # à modifier !!!!
    #     ellipse_filled=ndimage.binary_fill_holes(ellipse)
    #     filled_ellipses.append(ellipse_filled)
    #     filled_ellipses_np=np.array(filled_ellipses)
    # return filled_ellipses_np

#cv2.drawcontour(ellipse[index],)



# def filling_ellipses(image_to_fill):
#     #avec fill binary holes
#     '''
#     En partant des images 3D des ellipses, on crée une image 3D des ellipses remplies. 
#     Args:
#         ellipses (np.array): les ellipses à remplir
#     Return:
#         filled_ellipses_np (np.array): les ellipses remplies
#     ''' 
#     result = np.zeros_like(image_to_fill,dtype=np.uint8)
#     for index in range(len(image_tofill)):
#         contours,  = cv2.findContours(image_to_fill[index], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#         if len(contours)>0:
#             cv2.drawContours(cv2.drawContours(result, contours, -1, 1, -1))
#     return result

# #cv2.drawcontour(ellipse[index],)

In [11]:
def keeping_arteries_only(filled_ellipses, image):
    '''
    Le but est de garder l'image originale en la restreignant à l'intérieur des ellipses.
    Args:
        filled_ellipses (np.array): les ellipses remplies
        image (np.array): image d'origine 
    Return:
        list_arteries_only (np.array): l'image originale restreinte à l'intérieur des ellipses
    ''' 
    list_arteries_only=[]
    for index in range(len(filled_ellipses)):
        slice_ell=filled_ellipses[index]
        slice_im=image[index]
        arteries_only=np.where(slice_ell==1,slice_im, np.nan)
        list_arteries_only.append(arteries_only)
    return np.array(list_arteries_only)

In [12]:
def plotting_hist(images):
    '''
    En partant  des images 3D, on plot l'histogramme de chaque slice.
    Args:
        images (np.array)
    ''' 
    count=0
    for index in range(len(images)):
        slice_im=images[index]
        if 255 in slice_im and count <10:
            print(index,end=', ')
            # plt.hist(slice_im.flatten(), bins=255)
            # plt.title(index)
            # plt.ylim((1,20))
            # plt.xlim((1,256))
            # plt.show()
            # count+=1

# constructing the whole network

In [13]:
mask_list = np.load(Path("temp/mask_list.npy"))
range_to_do=2#len(mask_list)
info_all_masks, ellipses_network=construct_ellipses_network(mask_list,range_to_do)

1  out of  2


100%|██████████| 265/265 [00:46<00:00,  5.74it/s]


2  out of  2


100%|██████████| 230/230 [00:32<00:00,  7.02it/s]


In [14]:
# image_array = np.load(Path("temp/preprocessed_image.npy"))
# network_mask=construct_mask_network(mask_list,range_to_do)

100%|██████████| 2/2 [00:18<00:00,  9.07s/it]


to run : 4min

average testing

In [17]:
# network_arteries_only_no255=np.where(network_arteries_only==255, np.nan, network_arteries_only)
# network_mean_slice=[]
# print(len(network_arteries_only_no255))
# for index in range(len(network_arteries_only_no255)):
#     slice=network_arteries_only_no255[index]
#     network_mean_slice.append(np.nanmean(slice))

# moyenne_network = np.nanmean(network_mean_slice)
# std_network = np.nanstd(network_mean_slice)
# print("mean gray for all network: ", moyenne_network)
# print("std gray for all network: ",std_network)

# list_deviating_indices=[]
# for index in range(len(network_mean_slice)):
#     if network_mean_slice[index] < moyenne_network - 1.5*std_network:
#         list_deviating_indices.append(index)
# print(list_deviating_indices)

1532


Mean of empty slice


mean gray for all network:  65.96514419767912
std gray for all network:  10.093494560958149
[1149, 1150, 1152, 1153, 1154, 1170, 1357, 1358, 1359, 1395, 1396, 1397, 1398, 1399, 1400]


In [18]:
viewer = napari.Viewer()
image_layer = viewer.add_image(image_array, colormap="gray")
image_layer = viewer.add_image(ellipses_network, colormap='blue',blending="additive")
image_layer = viewer.add_image(network_mask, colormap='gray_r',blending="additive")
napari.run()